In [1]:
# Heat budget terms
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os

In [29]:
def heat_budget(filename):
    state0 = xr.open_dataset(filename) 
    state = state0.isel(Y = range(1,10)) # Remove the boundary walls in y direction
    Time = state.T.data
    pt = state.Temp.data
    u0 = state.U.data
    u = (u0[:, :, :, 1:] + u0[:, :, :, :-1]) / 2
    
    Cp = 4220  # Specific heat content J/kg/C
    rho = 1025 # Density
    # Grid areas
    Area = np.empty([50, 9])
    Area[:10, :] = 100
    Area[10:20, :] = 200
    Area[20:30, :] = 300
    Area[30:40, :] = 800
    Area[40:, :] = 1600
    
    H1 = np.empty(len(Time))
    H2 = np.empty(len(Time))
    pt_vol = np.empty(len(Time))
    for t in range(len(Time)):
        pt_t = pt[t,:,:,1:29]
        H1[t] = -Cp*rho*np.sum(u[t,:,:,28]*Area*(pt[t,:,:,28]))
        H2[t] = Cp*rho*np.sum(u[t,:,:,1]*Area*(pt[t,:,:,1]))
        pt_x = np.empty(28)
        for xl in range(28):
            pt_x[xl] = np.sum(pt_t[:,:,xl]*Area*200) # Theta*V
        pt_vol[t] = pt_x.sum()
    Hax = H1.mean()
    HR = H2.mean()
    Hst = np.gradient(pt_vol, Time).mean()*Cp*rho
    return (Hax, HR, Hst)

In [36]:
Hax, HR, Hst = heat_budget('/Users/weiyangbao/Documents/Model_outputs/nodrag_010/state_009.nc')

#state0 = xr.open_dataset('/Users/weiyangbao/Desktop/State/'+str(format(Hs,'03d'))+'/state_'+str(format(int(Qf/100),'03d'))+'.nc')
Hst-Hax-HR

1689991844.4836729